In [23]:
import numpy as np
import pandas as pd

In [24]:
def concordance_function(difference,indifference,preference,gain):
    if gain:
        if difference >= -indifference:
            return 1
        elif difference < -preference:
            return 0
        else:
            return (preference + difference) / (preference-indifference)
    else:
        if difference <= indifference:
            return 1
        elif difference > preference:
            return 0
        else:
            return (preference-difference) / (preference-indifference)


def calculate_marginal_concordance(alternatives, boundary_profiles, indifference_thresholds, preference_thresholds, gain,concordance_function=concordance_function):
    # Calculate marginal concordance for each alternative and boundary profile
    # Arguments:
    #   alternatives: pandas dataframe with alternatives
    #   boundary_profiles: pandas dataframe with boundary profiles
    #   indifference_thresholds: list of indifference thresholds for each boundary profile
    #   preference_thresholds: list of preference thresholds for each boundary profile
    #   gain: list of boolean values indicating whether the criterion is gain or cost
    # Returns:
    #   marginal_concordance: numpy array with marginal concordance in the shape of (number of criteria, number of alternatives, number of boundary profiles)

    marginal_concordance_alt_to_profile = np.zeros((len(boundary_profiles),len(alternatives),len(alternatives.columns),))
    marginal_concordance_profile_to_alt = np.zeros((len(boundary_profiles),len(alternatives),len(alternatives.columns),))

    for i in range(len(alternatives)):
        for j in range(len(boundary_profiles)):
            for k in range(len(alternatives.columns)):
                difference = alternatives.iloc[i,k] - boundary_profiles.iloc[j,k]
                
                indifference = indifference_thresholds[j][k]
                preference = preference_thresholds[j][k]
                marginal_concordance_alt_to_profile[j,i,k] = concordance_function(difference, indifference, preference, gain[k])
                marginal_concordance_profile_to_alt[j,i,k] = concordance_function(-difference, indifference, preference, gain[k])

    return marginal_concordance_alt_to_profile,marginal_concordance_profile_to_alt

def test_marginal_concordance():

    
    test_alternatives = pd.DataFrame([[90,86,46,30],
    [40,90,14,48],
    [94,100,40,36],
    [78,76,30,50],
    [60,60,30,30],
    [64,72,12,46],
    [62,88,22,48],
    [70,30,12,12]]).astype(float)
    test_boundary_profiles = pd.DataFrame( [[64,61,32,32],
[86,84,43,43]]).astype(float)
    test_indifference_thresholds = [[2,2,0,0],[3,2,0,0]]
    test_preference_thresholds = [[6,5,2,2],[7,8,2,2]]

    test_gains = [True,True,True,True]

    marginal_concordance_alt_to_profile, marginal_concordance_profile_to_alt = calculate_marginal_concordance(test_alternatives, test_boundary_profiles, test_indifference_thresholds, test_preference_thresholds, test_gains)

    concordance_matrix_alt_to_profile = np.array(
        [
        [1,1,1,0],
        [0,1,0,1],
        [1,1,1,1],
        [1,1,0,1],
        [0.5,1,0,0],
        [1,1,0,1],
        [1,1,0,1],
        [1,0,0,0]
        ],
    )
    concordance_matrix_profile_to_alt = np.array(
        [
        [0,0,0,1],
        [1,0,1,0],
        [0,0,0,0],
        [0,0,1,0],
        [1,1,1,1],
        [1,0,1,0],
        [1,0,1,0],
        [0,1,1,1]
        ]
    )
    assert np.allclose(marginal_concordance_alt_to_profile[0], concordance_matrix_alt_to_profile, atol=0.01)
    assert np.allclose(marginal_concordance_profile_to_alt[0], concordance_matrix_profile_to_alt, atol=0.01)

    return marginal_concordance_alt_to_profile, marginal_concordance_profile_to_alt

test_marginal_concordance()

(array([[[1. , 1. , 1. , 0. ],
         [0. , 1. , 0. , 1. ],
         [1. , 1. , 1. , 1. ],
         [1. , 1. , 0. , 1. ],
         [0.5, 1. , 0. , 0. ],
         [1. , 1. , 0. , 1. ],
         [1. , 1. , 0. , 1. ],
         [1. , 0. , 0. , 0. ]],
 
        [[1. , 1. , 1. , 0. ],
         [0. , 1. , 0. , 1. ],
         [1. , 1. , 0. , 0. ],
         [0. , 0. , 0. , 1. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 1. ],
         [0. , 1. , 0. , 1. ],
         [0. , 0. , 0. , 0. ]]]),
 array([[[0.        , 0.        , 0.        , 1.        ],
         [1.        , 0.        , 1.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 1.        , 0.        ],
         [1.        , 1.        , 1.        , 1.        ],
         [1.        , 0.        , 1.        , 0.        ],
         [1.        , 0.        , 1.        , 0.        ],
         [0.        , 1.        , 1.        , 1.        ]],
 
        [[0.75      , 1

In [25]:
def discordance_function(difference, veto, preference, gain):
    if gain:
        if difference <= -veto:
            return 1
        elif difference >= -preference:
            return 0
        else :
            return (-difference - preference) / (veto-preference)
    else:
        if difference >= veto:
            return 1
        elif difference <= preference:
            return 0
        else:
            return (veto - difference) / (veto-preference)

def calculate_marginal_discordance(alternatives,boundary_profiles,preference_thresholds,veto_thresholds,gain,discordance_function=discordance_function):

    marginal_discordance_alt_to_profile = np.zeros((len(boundary_profiles),len(alternatives),len(alternatives.columns),))
    marginal_discordance_profile_to_alt = np.zeros((len(boundary_profiles),len(alternatives),len(alternatives.columns),))

    for i in range(len(alternatives)):
        for j in range(len(boundary_profiles)):
            for k in range(len(alternatives.columns)):
                difference = alternatives.iloc[i,k] - boundary_profiles.iloc[j,k]
                
                veto = veto_thresholds[j][k]
                preference = preference_thresholds[j][k]
                marginal_discordance_alt_to_profile[j,i,k] = discordance_function(difference, veto, preference, gain[k])
                marginal_discordance_profile_to_alt[j,i,k] = discordance_function(-difference, veto, preference, gain[k])

    return marginal_discordance_alt_to_profile,marginal_discordance_profile_to_alt

def test_marginal_discordance():

    test_alternatives = pd.DataFrame([[90,86,46,30],
    [40,90,14,48],
    [94,100,40,36],
    [78,76,30,50],
    [60,60,30,30],
    [64,72,12,46],
    [62,88,22,48],
    [70,30,12,12]]).astype(float)
    test_boundary_profiles = pd.DataFrame( [[64,61,32,32],
[86,84,43,43]]).astype(float)
    test_veto_thresholds = [[20,24,np.inf,np.inf],[20,25,np.inf,np.inf]]
    test_preference_thresholds = [[6,5,2,2],[7,8,2,2]]

    test_gains = [True,True,True,True]

    discordance_alt_to_profile = np.array([
        [0,0,0,0],
        [1,0,0,0],
        [0,0,0,0],
        [0.07,0,0,0],
        [1,0.94,0,0],
        [1,0.23,0,0],
        [1,0,0,0],
        [0.69,1,0,0]
    ])
    discordance_profile_to_alt = np.array([
        [1,1,0,0],
        [0,1,0,0],
        [1,1,0,0],
        [0.57,0.52,0,0],
        [0,0,0,0],
        [0,0.31,0,0],
        [0,1,0,0],
        [0,0,0,0]
    ])

    marginal_discordance_alt_to_profile, marginal_discordance_profile_to_alt = calculate_marginal_discordance(test_alternatives, test_boundary_profiles, test_preference_thresholds, test_veto_thresholds,test_gains)

    assert np.allclose(marginal_discordance_alt_to_profile[1], discordance_alt_to_profile, atol=0.01)
    assert np.allclose(marginal_discordance_profile_to_alt[0], discordance_profile_to_alt, atol=0.01)

    return marginal_discordance_alt_to_profile, marginal_discordance_profile_to_alt
    
test_marginal_discordance()

(array([[[0.        , 0.        , 0.        , 0.        ],
         [1.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 1.        , 0.        , 0.        ]],
 
        [[0.        , 0.        , 0.        , 0.        ],
         [1.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.07692308, 0.        , 0.        , 0.        ],
         [1.        , 0.94117647, 0.        , 0.        ],
         [1.        , 0.23529412, 0.        , 0.        ],
         [1.        , 0.        , 0.        , 0.        ],
         [0.69230769, 1.        , 0.        , 0.        ]]]),
 array([[[1.        , 1.        , 0.        , 0.  

In [26]:
def calculate_comprehensive_concordance(concordance_matrix_alt_to_profile, concordance_matrix_profile_to_alt,weights):

    # weighted average of concordances for each alternative

    comprehensive_concordance_alt_to_profile = np.average(concordance_matrix_alt_to_profile, axis=2, weights=weights)
    comprehensive_concordance_profile_to_alt = np.average(concordance_matrix_profile_to_alt, axis=2, weights=weights)

    return comprehensive_concordance_alt_to_profile.T, comprehensive_concordance_profile_to_alt.T

def test_comprehensive_concordance():

    concordance_matrix_alt_to_profile, concordance_matrix_profile_to_alt = test_marginal_concordance()

    weights = [0.4,0.3,0.25,0.05]

    comprehensive_concordance_alt_to_profile, comprehensive_concordance_profile_to_alt = calculate_comprehensive_concordance(concordance_matrix_alt_to_profile, concordance_matrix_profile_to_alt,weights)
    return comprehensive_concordance_alt_to_profile, comprehensive_concordance_profile_to_alt

test_comprehensive_concordance()

(array([[0.95, 0.95],
        [0.35, 0.35],
        [1.  , 0.7 ],
        [0.75, 0.05],
        [0.5 , 0.  ],
        [0.75, 0.05],
        [0.75, 0.35],
        [0.4 , 0.  ]]),
 array([[0.05, 0.65],
        [0.65, 0.75],
        [0.  , 0.3 ],
        [0.25, 0.95],
        [1.  , 1.  ],
        [0.65, 0.95],
        [0.65, 0.85],
        [0.6 , 1.  ]]))

In [27]:
def calculate_outranking_credibility(comprehensive_concordance,marginal_discordance):
    # Returns array of outranking credibility for each alternative
    # outranking[alternative][boundary profile][0] = outranking credibility of alternative to boundary profile
    # outranking[alternative][boundary profile][1] = outranking credibility of boundary profile to alternative
    outrankings = np.zeros((len(comprehensive_concordance[0]),len(comprehensive_concordance[0][0]),2))
    for i in range(len(comprehensive_concordance[0])): # numver of alternatives
        for j in range(len(comprehensive_concordance[0][0])): # number of boundary profiles
            outranking = [comprehensive_concordance[0][i][j], comprehensive_concordance[1][i][j]]
            for k in range(len(marginal_discordance[0])): # number of criteria
                
                if comprehensive_concordance[0][i][j] < marginal_discordance[0][j][i][k]: # alt_to_profile
                    outranking[0] *= (1-marginal_discordance[0][j][i][k])/(1-comprehensive_concordance[0][i][j])
                if comprehensive_concordance[1][i][j] < marginal_discordance[1][j][i][k]: # profile_to_alt
                    outranking[1] *= (1-marginal_discordance[1][j][i][k])/(1-comprehensive_concordance[1][i][j])
            outrankings[i][j] = outranking
    
    return outrankings

def test_outranking_credibility():

    comprehensive_concordance = test_comprehensive_concordance()
    marginal_discordance = test_marginal_discordance()

    outrankings = calculate_outranking_credibility(comprehensive_concordance,marginal_discordance)
    test_outrankings = np.array([[0.95,0],[0.95,0.65]])
    assert np.allclose(outrankings[0], test_outrankings, atol=0.01)
    return outrankings
test_outranking_credibility()

array([[[0.95      , 0.        ],
        [0.95      , 0.65      ]],

       [[0.        , 0.        ],
        [0.        , 0.75      ]],

       [[1.        , 0.        ],
        [0.7       , 0.22689076]],

       [[0.75      , 0.09022556],
        [0.048583  , 0.95      ]],

       [[0.5       , 1.        ],
        [0.        , 1.        ]],

       [[0.75      , 0.65      ],
        [0.        , 0.95      ]],

       [[0.75      , 0.        ],
        [0.        , 0.85      ]],

       [[0.        , 0.6       ],
        [0.        , 1.        ]]])

In [28]:
def preference_aggregation(aPb,bPa):
    if aPb and not bPa:
        return 1 # a is preferred to b
    if not aPb and bPa:
        return -1 # b is preferred to a
    if aPb and bPa: 
        return 0 # indifferent
    if not aPb and not bPa:
        return None # incomparable

def transform_outranking_to_preference(outrankings,credibility_threshold=0.65):

    outranking_preference = np.zeros((len(outrankings),len(outrankings[0])))
    for i in range(len(outrankings)): # alternatives
        for j in range(len(outrankings[0])): # boundary profiles
            outranking_preference[i][j] = preference_aggregation(outrankings[i][j][0] >= credibility_threshold, outrankings[i][j][1] >= credibility_threshold)
    return outranking_preference

def test_transform_outranking_to_preference():
    outrankings = test_outranking_credibility()
    outranking_preference = transform_outranking_to_preference(outrankings)
    assert np.allclose(outranking_preference[:,0:1].flatten(), [[1,np.nan,1,1,-1,0,1,np.nan]], equal_nan=True)
    return outranking_preference

preference = test_transform_outranking_to_preference()
print(preference[:,0:1])
print(preference[:,1:2])

[[ 1.]
 [nan]
 [ 1.]
 [ 1.]
 [-1.]
 [ 0.]
 [ 1.]
 [nan]]
[[ 0.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]


In [29]:
def class_assignment(preference_matrix,num_classes, pessimistic=True):
    if pessimistic:
        # fill array of size len(preference_matrix) with max value of 2
        classes = np.full(len(preference_matrix),num_classes)

    else:
        classes = np.ones(len(preference_matrix))


    for j,a in enumerate(preference_matrix):
        if pessimistic:
            for i in range(len(a)-1,-1,-1):
                
                if a[i] >=0:
                    break
                classes[j]-=1
            
        else:
            for i in range(len(a)):
                if a[i] == -1:
                    break
                classes[j]+=1
                
    return classes

def test_class_assignment():

    preference = test_transform_outranking_to_preference()
    classes_pess = class_assignment(preference,3,True)
    classes_opt = class_assignment(preference,3,False)
    assert np.allclose(classes_pess, [3,1,3,2,1,2,2,1])
    assert np.allclose(classes_opt, [3,2,3,2,1,2,2,2])
    return classes_pess, classes_opt

test_class_assignment()

(array([3, 1, 3, 2, 1, 2, 2, 1]), array([3., 2., 3., 2., 1., 2., 2., 2.]))

In [30]:
def electre_tri_b(alternatives,boundary_profiles,criteria_is_gain,indifference_thresholds,preference_thresholds,veto_thresholds,weights,pessimistic=True):
    # Returns array of classes for each alternative

    # Calculate concordance and discordance matrices
    concordance_matrix_alt_to_profile, concordance_matrix_profile_to_alt = calculate_marginal_concordance(alternatives,boundary_profiles,indifference_thresholds,preference_thresholds,criteria_is_gain)
    discordance_matrix_alt_to_profile, discordance_matrix_profile_to_alt = calculate_marginal_discordance(alternatives,boundary_profiles,preference_thresholds,veto_thresholds,criteria_is_gain)

    # Calculate comprehensive concordance
    comprehensive_concordance = calculate_comprehensive_concordance(concordance_matrix_alt_to_profile,concordance_matrix_profile_to_alt,weights)

    # Calculate outranking credibility
    outrankings = calculate_outranking_credibility(comprehensive_concordance,[discordance_matrix_alt_to_profile,discordance_matrix_profile_to_alt])

    # Transform outranking credibility to preference
    outranking_preference = transform_outranking_to_preference(outrankings)

    # Assign classes
    classes = class_assignment(outranking_preference,len(boundary_profiles)+1,pessimistic)

    
    return classes

def test_electre_tri_b():
    
    test_alternatives = pd.DataFrame([[70,98,78,76],
    [44,51,23,46],
    [94,100,43,36],
    [78,76,30,50],
    [60,60,30,30],
    [64,72,12,46],
    [62,88,22,48],
    [70,30,12,12]]).astype(float)
    test_boundary_profiles = pd.DataFrame( [[64,61,32,32],
[86,84,43,43]]).astype(float)
    test_veto_thresholds = [[20,24,np.inf,np.inf],[20,25,np.inf,np.inf]]
    test_preference_thresholds = [[6,5,2,2],[7,8,2,2]]
    test_indifference_thresholds = [[2,2,0,0],[3,2,0,0]]
    test_gains = [True,True,True,True]
    test_weights = [0.4,0.3,0.25,0.05]

    classes = electre_tri_b(test_alternatives,test_boundary_profiles,test_gains,test_indifference_thresholds,test_preference_thresholds,test_veto_thresholds,test_weights)
    assert np.allclose(classes, [3,1,3,2,1,2,2,1])
    return classes

test_electre_tri_b() 

array([3, 1, 3, 2, 1, 2, 2, 1])

# hparams 

Must watch
Good movie
Meh
Maybe
Skip

In [31]:
boundary_profiles = pd.DataFrame([
    [100,100,100,100,100,100,100],
    [100,100,100,100,100,100,100],
    [100,100,100,100,100,100,100],
    [100,100,100,100,100,100,100],
])
# I feel indifferent when the difference is X
INDIFFERENCE_THRESHOLDS = [
    [1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1],
]
# I feel preference when the difference is X
PREFERENCE_THRESHOLDS = [
    [5, 5, 5, 5, 5, 5, 5],
    [5, 5, 5, 5, 5, 5, 5],
    [5, 5, 5, 5, 5, 5, 5],
    [5, 5, 5, 5, 5, 5, 5],
]
# A is always better than B if the difference is X and there is no question about that
VETO_THRESHOLDS = [
    [10, 10, 10, 10, 10, 10, 10],
    [10, 10, 10, 10, 10, 10, 10],
    [10, 10, 10, 10, 10, 10, 10],
    [10, 10, 10, 10, 10, 10, 10],
]
gain = [True, True, True, True, True, True, True]

WEIGHTS = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
boundary_profiles.columns = ['Acting','Plot','Pictures','Music','Sentiment', 'Critics Score', 'Oscars Won']

In [32]:
b0 = pd.DataFrame([INDIFFERENCE_THRESHOLDS[0],PREFERENCE_THRESHOLDS[0],VETO_THRESHOLDS[0]],columns=boundary_profiles.columns)
b0["threshold"] = ["indifference","preference","veto"]
b0 = b0.set_index("threshold")

b1 = pd.DataFrame([INDIFFERENCE_THRESHOLDS[1],PREFERENCE_THRESHOLDS[1],VETO_THRESHOLDS[1]],columns=boundary_profiles.columns)
b1["threshold"] = ["indifference","preference","veto"]
b1 = b1.set_index("threshold")

b2 = pd.DataFrame([INDIFFERENCE_THRESHOLDS[2],PREFERENCE_THRESHOLDS[2],VETO_THRESHOLDS[2]],columns=boundary_profiles.columns)
b2["threshold"] = ["indifference","preference","veto"]
b2 = b2.set_index("threshold")

print("BOUNDARY PROFILE 0\n\n",b0)
print("BOUNDARY PROFILE 1\n\n",b1)
print("BOUNDARY PROFILE 2\n\n",b2)

BOUNDARY PROFILE 0

               Acting  Plot  Pictures  Music  Sentiment  Critics Score  \
threshold                                                               
indifference       1     1         1      1          1              1   
preference         5     5         5      5          5              5   
veto              10    10        10     10         10             10   

              Oscars Won  
threshold                 
indifference           1  
preference             5  
veto                  10  
BOUNDARY PROFILE 1

               Acting  Plot  Pictures  Music  Sentiment  Critics Score  \
threshold                                                               
indifference       1     1         1      1          1              1   
preference         5     5         5      5          5              5   
veto              10    10        10     10         10             10   

              Oscars Won  
threshold                 
indifference           1  
preference

In [ ]:
test_alternatives = pd.DataFrame([[70,98,78,76],
[44,51,23,46],
[94,100,43,36],
[78,76,30,50],
[60,60,30,30],
[64,72,12,46],
[62,88,22,48],
[70,30,12,12]]).astype(float)
test_boundary_profiles = pd.DataFrame([[64,61,32,32], [86,84,43,43]]).astype(float)
test_veto_thresholds = [[20,24,np.inf,np.inf],[20,25,np.inf,np.inf]]
test_preference_thresholds = [[6,5,2,2],[7,8,2,2]]
test_indifference_thresholds = [[2,2,0,0],[3,2,0,0]]
test_gains = [True,True,True,True]
test_weights = [0.4,0.3,0.25,0.05]

classes = electre_tri_b(test_alternatives,test_boundary_profiles,test_gains,test_indifference_thresholds,test_preference_thresholds,test_veto_thresholds,test_weights)
